In [18]:
import os
import boto3
import pandas as pd

In [19]:
DB_ACCESS_KEY = pd.read_csv('DB_ACCESS_KEY.csv')
aws_access_key_id = DB_ACCESS_KEY['Access key ID'][0]
aws_secret_access_key = DB_ACCESS_KEY['Secret access key'][0]

In [20]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
table = dynamodb.Table('StockSentiment')
# keep scanning until we have all the data in the table
response = table.scan()
data = response['Items']
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])
# convert the data to a pandas dataframe
sentiment_ticker_list = pd.DataFrame(data)
# convert Date column to datetime
sentiment_ticker_list['Date'] = pd.to_datetime(sentiment_ticker_list['Date'])
# make the index the Date column
sentiment_ticker_list = sentiment_ticker_list.set_index('Date').sort_index(ascending=False)
sentiment_ticker_list

,ticker_sentiment_score,ticker_sentiment_label,Stock,source,url,relevance_score
Date,,,,,,
2023-03-27 10:00:00,0.181614,Somewhat-Bullish,RSSS,Benzinga,https://www.benzinga.com/pressreleases/23/03/n...,0.089129
2023-03-27 10:00:00,0.404327,Bullish,RAIL,Benzinga,https://www.benzinga.com/pressreleases/23/03/g...,0.500324
2023-03-27 10:00:00,0.19024,Somewhat-Bullish,GDVM,PennyStocks.com,https://pennystocks.com/featured/2023/03/27/3-...,0.026532
2023-03-27 07:26:12,0.446184,Bullish,FSR,Investing News Network,https://investingnews.com/all-electric-fisker-...,0.862645
2023-03-27 07:26:12,0.317112,Somewhat-Bullish,GOOG,Investing News Network,https://investingnews.com/all-electric-fisker-...,0.090989
...,...,...,...,...,...,...
2023-02-21 07:42:00,0.003875,Neutral,NWG,Reuters,https://www.reuters.com/markets/europe/hsbc-cu...,0.140076
2023-02-21 07:31:30,0.327663,Somewhat-Bullish,NIO,Benzinga,https://www.benzinga.com/news/23/02/30980568/n...,0.730954
2023-02-21 07:31:30,0.086742,Neutral,TSLA,Benzinga,https://www.benzinga.com/news/23/02/30980568/n...,0.472338


In [21]:
rds_key_pass = pd.read_csv('db_key_pass.csv')
username = rds_key_pass['ID'][0]
password = rds_key_pass['PASS'][0]
url = rds_key_pass['URL'][0]

KeyError: 'password'

In [15]:
# change column names to match RDS table
sentiment_ticker_list = sentiment_ticker_list.rename(columns={'Stock': 'stock', 'ticker_sentiment_score': 'sentiment',
                                                              'ticker_sentiment_label': 'sentiment_label', 'url': 'article_url'})

In [16]:
# sentiment_ticker_list.rename(columns={'Date': 'posting_time'}, inplace=True)
sentiment_ticker_list['time_published'] = sentiment_ticker_list.index
# sentiment_ticker_list.set_index('posting_time', inplace=True)
sentiment_ticker_list

,sentiment,sentiment_label,stock,source,article_url,relevance_score,time_published
Date,,,,,,,
2023-03-27 10:00:00,0.181614,Somewhat-Bullish,RSSS,Benzinga,https://www.benzinga.com/pressreleases/23/03/n...,0.089129,2023-03-27 10:00:00
2023-03-27 10:00:00,0.404327,Bullish,RAIL,Benzinga,https://www.benzinga.com/pressreleases/23/03/g...,0.500324,2023-03-27 10:00:00
2023-03-27 10:00:00,0.19024,Somewhat-Bullish,GDVM,PennyStocks.com,https://pennystocks.com/featured/2023/03/27/3-...,0.026532,2023-03-27 10:00:00
2023-03-27 07:26:12,0.446184,Bullish,FSR,Investing News Network,https://investingnews.com/all-electric-fisker-...,0.862645,2023-03-27 07:26:12
2023-03-27 07:26:12,0.317112,Somewhat-Bullish,GOOG,Investing News Network,https://investingnews.com/all-electric-fisker-...,0.090989,2023-03-27 07:26:12
...,...,...,...,...,...,...,...
2023-02-21 07:42:00,0.003875,Neutral,NWG,Reuters,https://www.reuters.com/markets/europe/hsbc-cu...,0.140076,2023-02-21 07:42:00
2023-02-21 07:31:30,0.327663,Somewhat-Bullish,NIO,Benzinga,https://www.benzinga.com/news/23/02/30980568/n...,0.730954,2023-02-21 07:31:30
2023-02-21 07:31:30,0.086742,Neutral,TSLA,Benzinga,https://www.benzinga.com/news/23/02/30980568/n...,0.472338,2023-02-21 07:31:30


In [17]:
import pymysql
from sqlalchemy import create_engine

# Establish connection to the MySQL database
conn = pymysql.connect(host=url, user=username, password=password, db='stock_data')

# Create a SQLAlchemy engine object
engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)

# Convert the pandas DataFrame to a MySQL table
sentiment_ticker_list.to_sql(name='Sentiments', con=engine, if_exists='append', index=False)

# Close the connection
conn.close()
